In [1]:
import numpy as np
import os
import pickle
import cv2
import sys
import time

In [2]:
def img_tointegral_img(image):
    s = np.zeros((image.shape[0],image.shape[1]))
    integral = np.zeros((image.shape[0],image.shape[1]))
    for i in range(len(image)):
        for j in range(len(image[0])):
            if(i-1 >=0):
                s[i][j] = s[i-1][j] + image[i][j]
            else:
                s[i][j] = image[i][j]
            if(j-1 >=0):
                integral[i][j] = integral[i][j-1] + s[i][j]
            else:
                integral[i][j] = s[i][j]
            
    return integral

In [3]:
def get_image_data(is_train):
    if(is_train == True):
        folder_name = 'trainset'
    else:
        folder_name = 'testset'
    
    faces_train = []
    for i in os.listdir('dataset/'+folder_name+'/faces/'):
        name = 'dataset/'+folder_name+'/faces/' + i
        image = cv2.imread(name, cv2.IMREAD_GRAYSCALE)
        integral = img_tointegral_img(image)
        faces_train.append(integral)
    faces_train = np.array(faces_train)
    non_faces_train = []
    for i in os.listdir('dataset/'+folder_name+'/non-faces/'):
        name = 'dataset/'+folder_name+'/non-faces/' + i
        image = cv2.imread(name, cv2.IMREAD_GRAYSCALE)

        integral = img_tointegral_img(image)
        non_faces_train.append(integral)
    non_faces_train = np.array(non_faces_train)
    
    num_pos = len(faces_train)
    num_neg = len(non_faces_train)
    
    labels = []
    for i in range(num_pos):
        labels.append(1)
    for i in range(num_neg):
        labels.append(0)
    labels = np.array(labels)
    
    train_integral_data = []
    for i in faces_train:
        train_integral_data.append(i)
    for i in non_faces_train:
        train_integral_data.append(i)
    train_integral_data = np.array(train_integral_data)
    return (train_integral_data, labels)

In [8]:
train_input_data, train_labels = get_image_data(True)

In [9]:
test_input_data, test_labels = get_image_data(False)

In [10]:
train_input_data.shape, train_labels.shape

((2499, 19, 19), (2499,))

In [11]:
test_input_data.shape, test_labels.shape

((2473, 19, 19), (2473,))

In [4]:
def calc_value(x1,x2,x3,x4,integral):
    temp_sum = 0
    if(x4[0]>=0 and x4[1]>=0):
        temp_sum += integral[x4[0],x4[1]]
    if(x1[0]>=0 and x1[1]>=0):
        temp_sum += integral[x1[0],x1[1]]
    if(x2[0]>=0 and x2[1]>=0):
        temp_sum -= integral[x2[0],x2[1]]
    if(x3[0]>=0 and x3[1]>=0):
        temp_sum -= integral[x3[0],x3[1]]
    return temp_sum

In [5]:
cnt = 0
feature_index_feature_map = {}

In [6]:
def calc_features(input_data):
    image = input_data[0]
    features1 = []
    for integral in input_data:
        sample = []
        for i in range(1,len(image)+1):
            for j in range(1,len(image[0])+1):
                for x in range(-1,len(image)):
                    for y in range(-1,len(image[0])):
                        x1 = [x,y]
                        x4 = [x+i,y+j]
                        x2 = [x,y+j]
                        x3 = [x+i,y]
                        if(x4[0]>=len(image) or x4[1]>=len(image[0])):
                            continue
                        f1_temp1 = calc_value(x1,x2,x3,x4,integral)
                        x1 = [x,y+j]
                        x4 = [x+i,y+2*j]
                        x2 = [x,y+2*j]
                        x3 = [x+i,y+j]
                        if(x4[0]>=len(image) or x4[1]>=len(image[0])):
                            continue
                        f1_temp2 = calc_value(x1,x2,x3,x4,integral)
                        sample.append(-f1_temp1+f1_temp2)

        features1.append(sample)
    features1 = np.array(features1)
    
    features2 = []
    for integral in input_data:
        sample = []
        for i in range(1,len(image)+1):
            for j in range(1,len(image[0])+1):
                for x in range(-1,len(image)):
                    for y in range(-1,len(image[0])):
                        x1 = [x,y]
                        x4 = [x+i,y+j]
                        x2 = [x+i,y]
                        x3 = [x,y+j]
                        if(x4[0]>=len(image) or x4[1]>=len(image[0])):
                            continue
                        f2_temp1 = calc_value(x1,x2,x3,x4,integral)
                        x1 = [x+i,y]
                        x4 = [x+2*i,y+j]
                        x2 = [x+2*i,y]
                        x3 = [x+i,y+j]
                        if(x4[0]>=len(image) or x4[1]>=len(image[0])):
                            continue
                        f2_temp2 = calc_value(x1,x2,x3,x4,integral)
                        sample.append(+f2_temp1-f2_temp2)
        features2.append(sample)
    features2 = np.array(features2)
    
    features3 = []
    for integral in input_data:
        sample = []
        for i in range(1,len(image)+1):
            for j in range(1,len(image[0])+1):
                for x in range(-1,len(image)):
                    for y in range(-1,len(image[0])):
                        x1 = [x,y]
                        x4 = [x+i,y+j]
                        x2 = [x,y+j]
                        x3 = [x+i,y]
                        if(x4[0]>=len(image) or x4[1]>=len(image[0])):
                            continue
                        f1_temp1 = calc_value(x1,x2,x3,x4,integral)
                        x1 = [x,y+j]
                        x4 = [x+i,y+2*j]
                        x2 = [x,y+2*j]
                        x3 = [x+i,y+j]
                        if(x4[0]>=len(image) or x4[1]>=len(image[0])):
                            continue
                        #print(x1,x2,x3,x4)
                        f1_temp2 = calc_value(x1,x2,x3,x4,integral)
                        x1 = [x,y+2*j]
                        x4 = [x+i,y+3*j]
                        x2 = [x,y+3*j]
                        x3 = [x+i,y+2*j]
                        if(x4[0]>=len(image) or x4[1]>=len(image[0])):
                            continue
                        #print(x1,x2,x3,x4)
                        f1_temp3 = calc_value(x1,x2,x3,x4,integral)
                        sample.append(+f1_temp1-f1_temp2+f1_temp3)
        features3.append(sample)
    features3 = np.array(features3)
    
    features4 = []
    for integral in input_data:
        sample = []
        for i in range(1,len(image)+1):
            for j in range(1,len(image[0])+1):
                for x in range(-1,len(image)):
                    for y in range(-1,len(image[0])):
                        x1 = [x,y]
                        x4 = [x+i,y+j]
                        x2 = [x+i,y]
                        x3 = [x,y+j]
                        if(x4[0]>=len(image) or x4[1]>=len(image[0])):
                            continue
                        f2_temp1 = calc_value(x1,x2,x3,x4,integral)
                        x1 = [x+i,y]
                        x4 = [x+2*i,y+j]
                        x2 = [x+2*i,y]
                        x3 = [x+i,y+j]
                        if(x4[0]>=len(image) or x4[1]>=len(image[0])):
                            continue
                        f2_temp2 = calc_value(x1,x2,x3,x4,integral)
                        x1 = [x+2*i,y]
                        x4 = [x+3*i,y+j]
                        x2 = [x+3*i,y]
                        x3 = [x+2*i,y+j]
                        if(x4[0]>=len(image) or x4[1]>=len(image[0])):
                            continue
                        f2_temp3 = calc_value(x1,x2,x3,x4,integral)
                        sample.append(+f2_temp1-f2_temp2+f2_temp3)
        features4.append(sample)
    features4 = np.array(features4)
    
    features5 = []
    for integral in input_data:
        sample = []
        for i in range(1,len(image)+1):
            for j in range(1,len(image[0])+1):
                for x in range(-1,len(image)):
                    for y in range(-1,len(image[0])):
                        x1 = [x,y]
                        x4 = [x+i,y+j]
                        x2 = [x+i,y]
                        x3 = [x,y+j]
                        if(x4[0]>=len(image) or x4[1]>=len(image[0])):
                            continue
                        f2_temp1 = calc_value(x1,x2,x3,x4,integral)
                        x1 = [x+i,y]
                        x4 = [x+2*i,y+j]
                        x2 = [x+2*i,y]
                        x3 = [x+i,y+j]
                        if(x4[0]>=len(image) or x4[1]>=len(image[0])):
                            continue
                        f2_temp2 = calc_value(x1,x2,x3,x4,integral)
                        x1 = [x,y+j]
                        x4 = [x+i,y+2*j]
                        x2 = [x+i,y+j]
                        x3 = [x,y+2*j]
                        if(x4[0]>=len(image) or x4[1]>=len(image[0])):
                            continue
                        f2_temp3 = calc_value(x1,x2,x3,x4,integral)
                        x1 = [x+i,y+j]
                        x4 = [x+2*i,y+2*j]
                        x2 = [x+i,y+2*j]
                        x3 = [x+2*i,y+j]
                        if(x4[0]>=len(image) or x4[1]>=len(image[0])):
                            continue
                        f2_temp4 = calc_value(x1,x2,x3,x4,integral)
                        sample.append(-f2_temp1+f2_temp2+f2_temp3-f2_temp4)
        features5.append(sample)
    features5 = np.array(features5)
    
    total_features = np.concatenate([features1,features2,features3,features4,features5],axis=1)
    return total_features

In [19]:
train_features = calc_features(train_input_data)

In [20]:
np.save('train_features.npy',train_features)
np.save('train_labels.npy',train_labels)

In [21]:
test_features = calc_features(test_input_data)

In [22]:
np.save('test_features.npy',test_features)
np.save('test_labels.npy',test_labels)

In [23]:
# loading train and test data

In [33]:
train_features = np.load('train_features.npy')
train_labels = np.load('train_labels.npy')

In [34]:
test_features = np.load('test_features.npy')
test_labels = np.load('test_labels.npy')

In [35]:
train_features.shape, test_features.shape

((2499, 63960), (2473, 63960))

In [36]:
# Initializing weights

In [37]:
num_pos = np.sum(train_labels)
num_neg = len(train_labels) - num_pos

In [38]:
weights = []
for i in range(num_pos):
    weights.append(1/float(2*num_pos))
for i in range(num_neg):
    weights.append(1/float(2*num_neg))
weights = np.array(weights)
weights = weights/np.sum(weights)

In [39]:
def weak_classifier(image_index,feature_index,threshold,polarity,is_train):
    if(is_train==True):
        if(polarity*train_features[image_index,feature_index] <  polarity*threshold):
            return 1
        else:
            return 0
    else:
        if(polarity*test_features[image_index,feature_index] <  polarity*threshold):
            return 1
        else:
            return 0

In [40]:
def one_round(weights):
    t_pos = 0
    t_neg = 0
    for i in range(len(train_labels)):
        if(train_labels[i]==1):
            t_pos += weights[i]
        else:
            t_neg += weights[i]
        
    all_clfs = []
    print(train_features.shape)
    for feature_index in range(train_features.shape[1]):
        
        feature_values = train_features[:,feature_index]
        sorted_feature_value_indices = np.argsort(feature_values)
        s_pos = 0
        s_neg = 0
        weights_pos_till_now,weights_neg_till_now = 0, 0
        values = [sys.maxsize,0,0] # error,feature_threshold,feature_polarity
        for i in sorted_feature_value_indices:
            cur_error = min(weights_neg_till_now+t_pos-weights_pos_till_now,weights_pos_till_now+t_neg-weights_neg_till_now)
            if(cur_error<values[0]):
                values[0] = cur_error
                values[1] = feature_values[i]
                if(s_pos > s_neg):
                    values[2] = 1
                else:
                    values[2] = -1
            if(train_labels[i]==1):
                s_pos += 1
                weights_pos_till_now += weights[i]
            else:
                s_neg += 1
                weights_neg_till_now += weights[i]
        all_clfs.append([feature_index,values[1],values[2]])
    
    best_clf_index,best_err = 0,sys.maxsize
    for clf_index,clf in enumerate(all_clfs):
        
        cur_err = 0.0
        for i in range(train_features.shape[0]):
            if(weak_classifier(i,clf[0],clf[1],clf[2],True)!=train_labels[i]):
                cur_err += weights[i]
        cur_err = cur_err/float(train_features.shape[0])
        if(cur_err < best_err):
            best_clf_index,best_err = clf_index, cur_err      
    return all_clfs[best_clf_index], best_err

In [41]:
def strong_clf(image_index,num_rounds,is_train):
    total = 0
    for round_index in range(num_rounds):
        if(weak_classifier(image_index,rounds_arr[round_index][0][0],rounds_arr[round_index][0][1],rounds_arr[round_index][0][2],is_train)==1):
            total += rounds_arr[round_index][1]
    if(total>= 0.5*sum(rounds_arr[:num_rounds,1])):
        return 1
    else:
        return 0

In [42]:
start_time = time.time()

total_rounds = []
for num_rounds_cascade in [1,5,5,10,20]:
    rounds = []
    for each_round in range(num_rounds_cascade):
        print(each_round)
        clf, error = one_round(weights)
        if(error==0):
            error = 0.001
            beta = error / float(1-error)
            alpha = np.log(1/float(beta))
            rounds.append([clf,alpha])
            break
        beta = error / float(1-error)
        for i in range(train_features.shape[0]):
            if(weak_classifier(i,clf[0],clf[1],clf[2],True)==train_labels[i]):
                weights[i] = weights[i] * (beta)
        weights = weights/np.sum(weights)
        alpha = np.log(1/float(beta))
        print(clf,alpha)
        rounds.append([clf,alpha])
    rounds_arr = np.array(rounds)
    total_rounds.append(rounds_arr)
    new_train_labels = []
    new_train_features = []
    cnt = 0
    for index,i in enumerate(train_labels):
        if(i==1):
            new_train_labels.append(i)
            new_train_features.append(train_features[index])
        if(i==0 and strong_clf(index,len(rounds_arr),True)==1):
            new_train_labels.append(i)
            new_train_features.append(train_features[index])
        if(i==0 and strong_clf(index,len(rounds_arr),True)==0):
            cnt += 1
    print("cnt: ",cnt)
    
    new_train_labels = np.array(new_train_labels)
    new_train_features = np.array(new_train_features)
    train_labels = new_train_labels
    train_features = new_train_features
    
    num_pos = np.sum(train_labels)
    num_neg = len(train_labels) - num_pos
    print(num_pos,num_neg)
    if(num_neg==0):
        break
    weights = []
    for i in range(num_pos):
        weights.append(1/float(2*num_pos))
    for i in range(num_neg):
        weights.append(1/float(2*num_neg))
    weights = np.array(weights)
    weights = weights/np.sum(weights)
    
end_time = time.time()

0
(2499, 63960)
[10142, 106.0, -1] 9.962126797991662
cnt:  1685
499 315
0
(814, 63960)
[23632, -102.0, 1] 8.351338958005064
1
(814, 63960)
[57037, -108.0, 1] 8.85593381435585
2
(814, 63960)
[60749, 178.0, 1] 13.752816722038773
3
(814, 63960)
[17474, -39.0, 1] 10.405906060005542
4
(814, 63960)
[29141, 105.0, -1] 14.258642400671638
cnt:  244
499 71
0
(570, 63960)
[28534, -164.0, 1] 8.38851507760307
1
(570, 63960)
[27370, 369.0, 1] 9.003587396393696
2
(570, 63960)
[31207, -379.0, 1] 12.950075022596868
3
(570, 63960)
[63774, 567.0, 1] 14.027967516500887
4
(570, 63960)
[2698, -82.0, 1] 12.396340647507712
cnt:  69
499 2
0
(501, 63960)
cnt:  2
499 0


In [43]:
print("Total time taken in seconds:",end_time-start_time)

Total time taken in seconds: 1731.4350264072418


In [44]:
total_rounds

[array([[list([10142, 106.0, -1]), 9.962126797991662]], dtype=object),
 array([[list([23632, -102.0, 1]), 8.351338958005064],
        [list([57037, -108.0, 1]), 8.85593381435585],
        [list([60749, 178.0, 1]), 13.752816722038773],
        [list([17474, -39.0, 1]), 10.405906060005542],
        [list([29141, 105.0, -1]), 14.258642400671638]], dtype=object),
 array([[list([28534, -164.0, 1]), 8.38851507760307],
        [list([27370, 369.0, 1]), 9.003587396393696],
        [list([31207, -379.0, 1]), 12.950075022596868],
        [list([63774, 567.0, 1]), 14.027967516500887],
        [list([2698, -82.0, 1]), 12.396340647507712]], dtype=object),
 array([[list([17290, 34.0, 1]), 6.906754778648554]], dtype=object)]

In [49]:
def strong_clf_eval(cascade_index,image_index,is_train):
    total = 0
    rounds_arr = total_rounds[cascade_index]
    for round_index in range(len(rounds_arr)):
        if(weak_classifier(image_index,rounds_arr[round_index][0][0],rounds_arr[round_index][0][1],rounds_arr[round_index][0][2],is_train)==1):
            total += rounds_arr[round_index][1]
    if(total>= 0.5*sum(rounds_arr[:,1])):
        return 1
    else:
        return 0

In [50]:
def strong_clf_cascade(image_index,is_train):
    for cascade_index in range(len(total_rounds)):
        if(strong_clf_eval(cascade_index,image_index,is_train)==0):
            return 0
    return 1

In [47]:
train_features = np.load('train_features.npy')
train_labels = np.load('train_labels.npy')

In [53]:

cur_acc = 0

for i in range(train_features.shape[0]):
    
        #if(strong_clf(i,num_rounds,True)==)
    if(strong_clf_cascade(i,True)==train_labels[i]):
        cur_acc += 1
cur_acc = cur_acc/float(train_features.shape[0])
print('Training acc:',cur_acc)

Training acc: 0.9555822328931572


In [52]:

cur_fp = 0

for i in range(train_features.shape[0]):
    
        #if(strong_clf(i,num_rounds,True)==)
    if(strong_clf_cascade(i,True)==1 and train_labels[i]==0):
        cur_fp += 1
cur_fp = cur_fp/float(train_features.shape[0])
print('Training False positive:',cur_fp)

Training False positive: 0.0


In [54]:

cur_fn = 0

for i in range(train_features.shape[0]):
    
    if(strong_clf_cascade(i,True)==0 and train_labels[i]==1):
        cur_fn += 1
cur_fn = cur_fn/float(train_features.shape[0])
print('Training False Negative:',cur_fn)

Training False Negative: 0.04441776710684274


In [55]:

cur_acc = 0

for i in range(test_features.shape[0]):
    
        #if(strong_clf(i,num_rounds,True)==)
    if(strong_clf_cascade(i,False)==test_labels[i]):
        cur_acc += 1
cur_acc = cur_acc/float(test_features.shape[0])
print('Testing acc:',cur_acc)

Testing acc: 0.8277395875454913


In [56]:

cur_fp = 0

for i in range(test_features.shape[0]):
    
        #if(strong_clf(i,num_rounds,True)==)
    if(strong_clf_cascade(i,False)==1 and test_labels[i]==0):
        cur_fp += 1
cur_fp = cur_fp/float(test_features.shape[0])
print('Testing False positive:',cur_fp)

Testing False positive: 0.0028305701577031944


In [57]:

cur_fn = 0

for i in range(test_features.shape[0]):
    
        #if(strong_clf(i,num_rounds,True)==)
    if(strong_clf_cascade(i,False)==0 and test_labels[i]==1):
        cur_fn += 1
cur_fn = cur_fn/float(test_features.shape[0])
print('Testing False positive:',cur_fn)

Testing False positive: 0.1694298422968055
